In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import visdom

In [70]:
#hihello를 RNN으로 구현하기
input_data='hihello'
char_list=set(input_data)
print(char_list)


{'h', 'i', 'l', 'o', 'e'}


In [4]:
vocabulary={}
for _,i in enumerate(char_list):
    result=np.zeros(len(char_list))
    result[_]=1
    vocabulary[i]=result
print(vocabulary)

{'h': array([1., 0., 0., 0., 0.]), 'i': array([0., 1., 0., 0., 0.]), 'l': array([0., 0., 1., 0., 0.]), 'o': array([0., 0., 0., 1., 0.]), 'e': array([0., 0., 0., 0., 1.])}


In [7]:
input_data_one_hot_coded=[]
for i in input_data:
    input_data_one_hot_coded.append(vocabulary[i])
input_data_one_hot_coded=np.array(input_data_one_hot_coded)
print(input_data_one_hot_coded)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [15]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size):
        super().__init__()
        self.i2h=nn.Linear(input_size,hidden_size)
        self.h2h=nn.Linear(hidden_size,hidden_size)
        self.h2o=nn.Linear(hidden_size,input_size) #input_size와 output size를 동일하게 하였다.
        self.activation=nn.Tanh()
    def forward(self,input_data,hidden_state):
        z=(self.i2h.forward(input_data)+self.h2h.forward(hidden_state))
        hidden=self.activation.forward(z)
        predict=self.h2o(hidden)
        return predict,hidden
        

In [54]:
model=RNN(5,3)
optimizer=torch.optim.Adam(model.parameters(),0.01)
cost_list=[]
for i in range(1000):
    total_cost=0
    hidden_state=torch.zeros(3)
    optimizer.zero_grad()
    for _,j in enumerate(input_data_one_hot_coded[:-1]):
        x=torch.FloatTensor(input_data_one_hot_coded[_])
        y=torch.LongTensor(input_data_one_hot_coded[_+1])
        predict,hidden_state=model.forward(x,hidden_state)
        cost=F.cross_entropy(predict.unsqueeze(0),y.argmax().unsqueeze(0))
        total_cost+=cost
    total_cost.backward()
    optimizer.step()
    cost_list.append(total_cost.item())
    if i%100==0:
        print(total_cost.item())
        

11.210399627685547
10.217915534973145
9.44580078125
8.517879486083984
7.378930568695068
6.270646572113037
5.27294397354126
4.429452419281006
3.7596383094787598
3.2391421794891357
2.838336706161499
2.5160300731658936
2.230282783508301
1.9606225490570068
1.7157304286956787
1.5020077228546143
1.3185632228851318
1.1621192693710327
1.029186487197876
0.9162663817405701
0.8199894428253174
0.7374290227890015
0.6661683320999146
0.6042280793190002
0.5499783158302307
0.5020822286605835
0.4594545066356659
0.4212336838245392
0.38675981760025024
0.35555848479270935
0.32731199264526367
0.3018084764480591
0.27888017892837524
0.2583501935005188
0.24001207947731018
0.22363317012786865
0.20897486805915833
0.1958138644695282
0.1839512139558792
0.173217311501503
0.16346777975559235
0.1545809656381607
0.14645427465438843
0.13899999856948853
0.13214311003684998
0.12581783533096313
0.11996909230947495
0.11454824358224869
0.10951308906078339
0.10482583194971085
0.10045395791530609
0.09636886417865753
0.0925449

In [34]:
F.cross_entropy(torch.tensor([[1,0,0,0.]]),torch.tensor([1])) #2d 1d

tensor(1.7437)

In [69]:
#predict
output='h'
hiden_state=torch.zeros(3)
for _,j in enumerate(input_data_one_hot_coded[:-1]):
    x=torch.FloatTensor(input_data_one_hot_coded[_])
    predict,hidden_state=model.forward(x,hidden_state)
    output+=list(char_list)[predict.argmax().item()]
print(output)

hihello


In [74]:
#RNN Basics
input_size=5
hidden_size=3
rnn=nn.RNN(input_size,hidden_size)
outputs,hidden_state=rnn.forward(torch.FloatTensor(input_data_one_hot_coded).unsqueeze(0)) #input data shape : (batch size, seq length, # of features)

In [81]:
print(result)

[array([1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1.]), array([0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0.]), array([0., 0., 0., 1., 0.]), array([1., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0.])]


In [86]:
#batch size와 seq length는 automatically calculate 된다.
input_data_2=['hihello','ihelloh','hellohi']
input_data_2_one_hot_coded=[]
for i in input_data_2:
    result=[]
    for j in i:
        result.append(vocabulary[j])
    input_data_2_one_hot_coded.append(result)
input_data_2_one_hot_coded=np.array(input_data_2_one_hot_coded)
print(input_data_2_one_hot_coded)
print(input_data_2_one_hot_coded.shape) #3,7,5

[[[1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [1. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]]]
(3, 7, 5)


In [87]:
input_data_2_one_hot_coded=torch.FloatTensor(input_data_2_one_hot_coded)
rnn=nn.RNN(5,4)
outputs,hidden_state=rnn.forward(input_data_2_one_hot_coded)
print(outputs) # 3, 7, 4
print(hidden_state) #1,7,4 

tensor([[[-0.0313,  0.5725, -0.4446,  0.4473],
         [-0.1808,  0.0325,  0.0327,  0.5436],
         [-0.0313,  0.5725, -0.4446,  0.4473],
         [-0.4741,  0.3437, -0.4145,  0.4427],
         [-0.2642,  0.4076, -0.5945,  0.5804],
         [-0.2642,  0.4076, -0.5945,  0.5804],
         [-0.6864,  0.1600, -0.5862,  0.6096]],

        [[-0.2404,  0.4622, -0.1179,  0.7261],
         [-0.3406,  0.6117, -0.5382,  0.4739],
         [-0.5210,  0.6783, -0.5315,  0.6566],
         [-0.5056,  0.5430, -0.7265,  0.7268],
         [-0.4431,  0.6907, -0.7472,  0.7375],
         [-0.7804,  0.5212, -0.7415,  0.7575],
         [-0.4321,  0.6643, -0.6957,  0.6057]],

        [[-0.3734,  0.7248, -0.5593,  0.6248],
         [-0.6075,  0.6266, -0.5813,  0.6924],
         [-0.5495,  0.6578, -0.7441,  0.7974],
         [-0.5429,  0.7011, -0.7826,  0.7818],
         [-0.8139,  0.5919, -0.7688,  0.8199],
         [-0.4661,  0.7459, -0.7154,  0.7132],
         [-0.4067,  0.4533, -0.2654,  0.7773]]], grad_fn

In [90]:
#seq len을 다르게 한다면?
#batch size와 seq length는 automatically calculate 된다.
input_data_3=['hihello','ihello','hlohi']
input_data_3_one_hot_coded=[]
for i in input_data_3:
    result=[]
    for j in i:
        result.append(vocabulary[j])
    input_data_3_one_hot_coded.append(result)
input_data_3_one_hot_coded=np.array(input_data_3_one_hot_coded)
print(input_data_3_one_hot_coded)
print(input_data_3_one_hot_coded.shape) #3,

[list([array([1., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0.]), array([1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1.]), array([0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0.]), array([0., 0., 0., 1., 0.])])
 list([array([0., 1., 0., 0., 0.]), array([1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1.]), array([0., 0., 1., 0., 0.]), array([0., 0., 1., 0., 0.]), array([0., 0., 0., 1., 0.])])
 list([array([1., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0.]), array([0., 0., 0., 1., 0.]), array([1., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0.])])]
(3,)


In [91]:
input_data_3_one_hot_coded=torch.FloatTensor(input_data_3_one_hot_coded)
rnn=nn.RNN(5,4)
outputs,hidden_state=rnn.forward(input_data_3_one_hot_coded)
print(outputs) # 3, 7, 4
print(hidden_state) #1,7,4  #안되네...

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.